### Notes
We might need to remove the common words in the corpus beside the stop words as those are common cross all corpus

In [1]:
import csv
import re as re

import nltk as nltk
import pandas as pd
from nltk.corpus import words
from pandas import DataFrame

nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

utterances = pd.read_csv('./temp_data/utterances.csv')
utterances['actionbody'] = utterances['actionbody'].astype(str)
print(f'total recods {len(utterances)}')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/jovyan/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


total recods 30108


In [2]:
# https://www.ibm.com/think/topics/stemming-lemmatization#:~:text=The%20practical%20distinction%20between%20stemming,be%20found%20in%20the%20dictionary.
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
 
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:         
        return wordnet.NOUN
       
def lemmatize_passage(text):
    words = re.split(r'([\s,"\'\.\[\]\(\)])',text)
    pos_tags = pos_tag(words)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return lemmatized_words
 
text = "after looking at the change history provided, we found that the issue occurred because the user was able to approve a batch with $340 that was already approved, which should not be allowed."
result = lemmatize_passage(text)
 
print("Original:", text)
print("Lemmatized:", result)

Original: after looking at the change history provided, we found that the issue occurred because the user was able to approve a batch with $340 that was already approved, which should not be allowed.
Lemmatized: ['after', ' ', 'look', ' ', 'at', ' ', 'the', ' ', 'change', ' ', 'history', ' ', 'provide', ',', '', ' ', 'we', ' ', 'find', ' ', 'that', ' ', 'the', ' ', 'issue', ' ', 'occur', ' ', 'because', ' ', 'the', ' ', 'user', ' ', 'be', ' ', 'able', ' ', 'to', ' ', 'approve', ' ', 'a', ' ', 'batch', ' ', 'with', ' ', '$340', ' ', 'that', ' ', 'be', ' ', 'already', ' ', 'approve', ',', '', ' ', 'which', ' ', 'should', ' ', 'not', ' ', 'be', ' ', 'allow', '.', '']


In [3]:
dic_words = {}
for w in words.words():
    dic_words.update({w:''})

len(dic_words)

235892

In [4]:
print(WordNetLemmatizer().lemmatize('thinking', wordnet.VERB))
print(WordNetLemmatizer().lemmatize('thinking', wordnet.ADJ))

think
thinking


In [5]:
vocabulary = {}
for idx, row in utterances.iterrows():
    b_words = lemmatize_passage(row['actionbody'])
    for w in b_words:
        if w not in dic_words:
            vocabulary.update({w:''})

print('create vocabulary DF')            
voc_df = DataFrame(columns=['word'])
voc_df['word'] = vocabulary.keys()
voc_df.set_index('word', inplace=True)
voc_df.sort_values(by=['word'], ascending=True, inplace=True)

print(f'size of vocabulary {len(voc_df)}')

create vocabulary DF
size of vocabulary 5333


In [6]:
masked_uttr_df = pd.read_csv('./temp_data/utterances_voc_20250316.csv')
masked_uttr_df.set_index('word',inplace=True)
len(masked_uttr_df)

26820

In [7]:
joined_voc_df = voc_df.join(masked_uttr_df, on='word')
joined_voc_df.to_csv('./temp_data/vocabulary.csv',quoting=csv.QUOTE_ALL)
len(joined_voc_df)

5333